# 전환율을 이용해서 실질월세 계산

In [1]:
import pandas as pd
from APICall import HouseType

In [33]:
def calc_newRent(houseType: HouseType, year: int) -> pd.DataFrame:
    FileName = f"data/{houseType.value}_{year}.csv"
    df = pd.read_csv(FileName)

    ChangeRateFileName = f"data/ChangeRate_SDMG_{houseType.value}.csv"
    ChangeRate = pd.read_csv(ChangeRateFileName)

    df["YM"] = df["dealDate"] // 100
    merged = pd.merge(df, ChangeRate, left_on="YM", right_on="Date", how="left")

    merged["monthlyRent"] = merged["monthlyRent"].astype(str).str.replace(',', '').astype(float)
    merged["deposit"] = merged["deposit"].astype(str).str.replace(',', '').astype(float)
    merged["calcedRent"] = merged["monthlyRent"] + merged["deposit"]*merged["ChangeRate"]/100
    
    merged["RPA"] = merged["calcedRent"] / merged["excluUseAr"]
    merged["RPA"] = merged["RPA"].round(2)

    merged = merged.drop(columns=["YM", "Date"])
    columns = ["dealDate", "deposit", "monthlyRent", "ChangeRate", "calcedRent", "excluUseAr", "RPA"]
    for col in merged.columns:
        if col not in columns:
            columns.append(col)
    merged = merged[columns]
    print(merged)
    merged.to_csv("data/tmp.csv")
    

In [34]:
calc_newRent(HouseType.Apt, 2010)

      dealDate  deposit  monthlyRent  ChangeRate  ...  sggCd  sggNm  umdNm  useRRRight
0     20110107   5000.0         30.0     7.20000  ...  11410   서대문구    연희동            
1     20110107   5000.0         30.0     7.20000  ...  11410   서대문구    연희동            
2     20110108   2000.0         65.0     7.20000  ...  11410   서대문구    대신동            
3     20110110   8000.0          0.0     7.20000  ...  11410   서대문구    창천동            
4     20110110   1000.0         55.0     7.20000  ...  11410   서대문구    연희동            
...        ...      ...          ...         ...  ...    ...    ...    ...         ...
1064  20191226  19000.0          0.0     3.71594  ...  11410   서대문구    창천동            
1065  20191228  10000.0         35.0     3.71594  ...  11410   서대문구    대현동            
1066  20191228   1000.0         85.0     3.71594  ...  11410   서대문구    대현동            
1067  20191231   5000.0         75.0     3.71594  ...  11410   서대문구    대현동            
1068  20191231  18500.0          0.0     3.